In [1]:
import os
import sys
import pandas as pd
src_dir = os.path.abspath('../src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [2]:
import prompts
from llm import LLM
from utils import md_table_to_df
from save_to_google import save_to_gsheet
from tgbot import save_message_to_file

In [3]:
test_message = {
    "date": "2025-04-15T19:02:21+03:00",
    "date_utc": "2025-04-15T16:02:21+00:00",
    "tg_message_id": 13,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": None,
    "raw_text": "Пахота зяби под мн тр\nПо Пу 13/540\nОтд 12 13/273\n\nДиск сах св\nПо Пу 70/1004\nОтд 17 70/302\n\nПредп культ под оз пш\nПо Пу 91/1403\nОтд 11 45/373\nОтд 12 46/363"
}

In [4]:
test_message2 = {
    "date": "2025-04-15T19:02:21+03:00",
    "date_utc": "2025-04-15T16:02:21+00:00",
    "tg_message_id": 13,
    "our_message_id": 1,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": None,
    "raw_text": "Бери 15 сотрудников и едьте по адресу Новороссийская 86\n там проведите свои измерения плиз"}

In [5]:
test_message3 = {
    "date": "2025-04-15T19:46:34+03:00",
    "date_utc": "2025-04-15T16:46:34+00:00",
    "tg_message_id": 187,
    "our_message_id": 3,
    "sender": {
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    },
    "reply_to_message_id": None,
    "raw_text": "С 16:30 до 19"
}

In [6]:
params = {
    "raw_message": test_message,
}

filled_prompt = prompts.AGRO_PROMPT.format(**params)
print(filled_prompt)


    <Инструкция>
    Сложность состоит в том, что
    в своих отчетах агрономы используют различные сокращения, а в итоговой таблице необходимо указать полное название работы.
    Чтобы ты смог выполнить эту задачу, я дам тебе справочную информацию (полные названия работ и всех наименовай, которые встретятся в отчетах сотрудников). Она будет приведена ниже.
    В итоговой таблице 6 обязательных столбцов, соответственно, в отчете сотрудников должны присутствовать как минимум 6 наименований. Отдельно отмечу, что в сообщении чаще всего может быть сразу несколько описаний работ,
    то есть агроном мог выполнять несколько работ за день и описать их в одном сообщении. Обычно в таком случае эти описания разделены пустыми строками, но могут и не быть. Если описаний несколько, то дополнительно сообщие мне об этом и укажи точное количество.
    В этом случае ты должен будешь указать все 6 обязательных полей для каждой работы.
    Как я уже сказал, обязательных полей всего 6, но всего их 8.


 

In [7]:
llm = LLM()

In [8]:
md_table = await llm.call_yandex_gpt(filled_prompt)

2025-04-17T10:58:08.900194+0300 - llm - INFO - {'result': {'alternatives': [{'message': {'role': 'assistant', 'text': '| Дата       | Подразделение | Операция                    | Культура                   | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |\n|------------|----------------|-----------------------------|----------------------------|-------------|------------------------|----------------|------------------|\n| 04/15/2025 | АОР            | Пахота                      | Многолетние травы текущего года | 13          | 540                    |                |                  |\n| 04/15/2025 | АОР            | Дискование                  | Свекла сахарная            | 70          | 1004                   |                |                  |\n| 04/15/2025 | АОР            | Предпосевная культивация    | Пшеница озимая товарная    | 91          | 1403                   |                |                  |\n\nКоличество найденных операций: 3'}, 'statu

In [9]:
md_table

'| Дата       | Подразделение | Операция                    | Культура                   | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |\n|------------|----------------|-----------------------------|----------------------------|-------------|------------------------|----------------|------------------|\n| 04/15/2025 | АОР            | Пахота                      | Многолетние травы текущего года | 13          | 540                    |                |                  |\n| 04/15/2025 | АОР            | Дискование                  | Свекла сахарная            | 70          | 1004                   |                |                  |\n| 04/15/2025 | АОР            | Предпосевная культивация    | Пшеница озимая товарная    | 91          | 1403                   |                |                  |\n\nКоличество найденных операций: 3'

In [10]:
df = md_table_to_df(md_table)
df

,Дата,Подразделение,Операция,Культура,"За день, га","С начала операции, га","Вал за день, ц","Вал с начала, ц"
0,2025-04-15,АОР,Пахота,Многолетние травы текущего года,13,540,NaN,NaN
1,2025-04-15,АОР,Дискование,Свекла сахарная,70,1004,NaN,NaN
2,2025-04-15,АОР,Предпосевная культивация,Пшеница озимая товарная,91,1403,NaN,NaN


In [11]:
save_to_gsheet(df)

TypeError: save_to_gsheet() missing 1 required positional argument: 'message_file_path'